In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json, lxml
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
# Plotting 
import matplotlib.cm as cm
import matplotlib.colors as colors
# Clustering
from sklearn.cluster import KMeans
# BS for wiki scrape
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [62]:
import folium

In [63]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source)

table_data = soup.find('div', class_='mw-parser-output')
table = table_data.table.tbody

columns = ['Postal Code', 'Borough', 'Neighbourhood']
data = dict({key:[]*len(columns) for key in columns})

for row in table.find_all('tr'):
    for i,column in zip(row.find_all('td'),columns):
        i = i.text
        i = i.replace('\n', '')
        data[column].append(i)

df = pd.DataFrame.from_dict(data=data)[columns]
print(df.shape)
df.head()

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [64]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop = True)
print('After dropping "Not assigned" rows: ',df.shape)
print('Neighbourhood is "Not assigned" but borough has value: ', 
      df[df['Neighbourhood'] == 'Not assigned'].shape[0])

After dropping "Not assigned" rows:  (103, 3)
Neighbourhood is "Not assigned" but borough has value:  0


In [65]:
p, b, n = [], [], []
for postcode, borough, neigh in zip(df['Postal Code'], df['Borough'], df['Neighbourhood']):
    p.append(postcode)
    b.append(borough)
    if neigh == 'Not assigned':
        n.append(borough)
    else:
        n.append(neigh)

df = pd.DataFrame({'Postal Code': p, 'Borough': b, 'Neighbourhood':n})[columns]
print(df.shape)
df.head()

(103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [66]:
postcodes = df['Postal Code'].values
boroughs = df['Borough'].values
neighs = df['Neighbourhood'].values

dic = dict({(key1,key2): [] for key1, key2 in zip(postcodes, boroughs)})
print('Number of keys in the dictionary are: ', len(dic.keys()))

for postcode, boroughs, neigh in zip(postcodes,boroughs, neighs):
    key = (postcode, borough)
    dic[key].append(neigh)

df = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighbourhood'])
for key, value in dic.items():
    postcode, borough, neig = key[0], key[1], value
    neig = ', '.join(neig)
    df = df.append({'Postal Code': postcode,
                     'Borough': borough,
                     'Neighbourhood': neig}, ignore_index = True)
print('shape = ', df.shape)
df.head(10)

Number of keys in the dictionary are:  103


KeyError: ('M3A', 'Etobicoke')

In [12]:
print('Getting latLong for postcodes')

Getting latLong for postcodes


In [67]:
import geocoder

In [52]:
lats, lons = [], []
count = 0
for postal_code in df['Postal Code'].values:
     lat_lng_coords = None

In [53]:
df = pd.read_csv (r'C:\Users\xxx Laptop\Desktop\Geospatial_Coordinates.csv')
print (df)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
..          ...        ...        ...
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

[103 rows x 3 columns]


In [54]:
try:
    df['Latitude'] = lats
    df['Longitude'] = lons
except:
    latlon = pd.read_csv('Geospatial_Coordinates.csv')
    df = pd.merge(df, latlon, how= 'inner', on = 'Postal Code')
    
print(df.shape)
df.head(10)

(103, 5)


,Postal Code,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M1B,43.806686,-79.194353,43.806686,-79.194353
1,M1C,43.784535,-79.160497,43.784535,-79.160497
2,M1E,43.763573,-79.188711,43.763573,-79.188711
3,M1G,43.770992,-79.216917,43.770992,-79.216917
4,M1H,43.773136,-79.239476,43.773136,-79.239476
5,M1J,43.744734,-79.239476,43.744734,-79.239476
6,M1K,43.727929,-79.262029,43.727929,-79.262029
7,M1L,43.711112,-79.284577,43.711112,-79.284577
8,M1M,43.716316,-79.239476,43.716316,-79.239476
9,M1N,43.692657,-79.264848,43.692657,-79.264848
